In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-4.1-nano-2025-04-14"

In [3]:
client = OpenAI(
    api_key = OPENAI_API_KEY
)

In [ ]:
# basic text generation using openai api

prompt = "list our the meaning of sanjay in 5 words"

response = client.responses.create(
    model = MODEL,
    input = prompt
)

result = response.output[0].content[0].text
print(f"prompt : {prompt}")
print(f"Response : {result}")

prompt : list our the meaning of sanjay in 5 words
Response : Sure! Here are five words that capture the meaning of "Sanjay":

1. Victory
2. Triumph
3. Conqueror
4. Winner
5. Lord (or leader)


In [ ]:
# controlled response
output_tokens = 150
prompt = """

Explain **what is all about "Attenention is all You Need" Paper** in a clear and detailed manner. Please ensure the explanation is comprehensive but concise enough to fit within the maximum output tokens specified below.  
**Note:** Limit your response to "{output_tokens}" tokens.

"""

response = client.responses.create(
    model = MODEL,
    input = prompt,
    temperature=0.3,
    # top_p=0.9,
    max_output_tokens = output_tokens
)

In [26]:

result = response.output[0].content[0].text
# print(f"prompt : {prompt}")
print(f"Response : {result}")


Response : Certainly! The "Attention Is All You Need" paper, introduced by Vaswani et al. in 2017, revolutionized natural language processing (NLP) by proposing a new model architecture called the Transformer. Here's a clear, comprehensive overview:

### Core Idea:
The paper argues that traditional sequence models like RNNs and LSTMs, which process data sequentially, have limitations in capturing long-range dependencies and are computationally intensive. Instead, they introduce a model that relies solely on **attention mechanisms**, eliminating recurrence and convolutions.

### Key Components:
1. **Self-Attention Mechanism:**
   - Allows the model to weigh the importance of different words in a sentence relative to each other.
   - For each word,


In [ ]:
# Image to text

image_url = "https://4.img-dpreview.com/files/p/E~TS590x0~articles/3925134721/0266554465.jpeg"

response = client.responses.create(
    model = MODEL,
    input = [{
        "role" : "user",
        "content" : [
            {"type" : "input_text", "text" :"what is this image about?"},
            {
                "type" : "input_image",
                "image_url" : image_url
            }
        ]
    }]
)


In [30]:
result = response.output[0].content[0].text
print(f"Image URL : {image_url}")
print(f"Response : {result}")

Image URL : https://4.img-dpreview.com/files/p/E~TS590x0~articles/3925134721/0266554465.jpeg
Response : The image shows a bright yellow bird perched on a branch among green leaves. The bird appears to be in a natural outdoor setting, likely in a forest or garden. The image captures the bird in a calm and natural pose, highlighting its vibrant coloration.


In [35]:
# Text to speech using OpenAI API

speech_file_path = "aivoice.mp3"

with client.audio.speech.with_streaming_response.create(
    model = "gpt-4o-mini-tts",
    voice = "coral",
    input = "I love my mom and dad. They are the best parents in the world. and I am so gratefull for them",
    instructions="Speak in a cheerful and positive tone",
) as response:
    response.stream_to_file(speech_file_path)

In [36]:
# speech to text

audio_file = open("aivoice.mp3", "rb")

transcription = client.audio.transcriptions.create(
    model = "gpt-4o-transcribe",
    file = audio_file
)


In [37]:
print(transcription.text)

I love my mom and dad. They are the best parents in the world, and I'm so grateful for them.


In [16]:
import requests
import json

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

input_messages = [{"role": "user", "content": "What's the weather like in Chennai today?"}]

response = client.responses.create(
    model="gpt-4.1-mini",
    input=input_messages,
    tools=tools,
)

In [17]:
tool_call = response.output[0]
args = json.loads(tool_call.arguments)

result = get_weather(args['latitude'], args["longitude"])
print(result)

33.7


In [18]:
input_messages.append(tool_call)
input_messages.append({
    "type" : "function_call_output",
    "call_id" : tool_call.call_id,
    "output" : str(result)
})

response_2 = client.responses.create(
    model = MODEL,
    input = input_messages,
    tools = tools
)


In [19]:
print(response_2.output[0].content[0].text)

The temperature in Chennai today is approximately 33.7°C.


In [20]:
import requests
import json

# This is the actual tool (function) you expose to the LLM
def get_weather(latitude, longitude):
    # Makes a real API request to Open-Meteo to get current weather data
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast"
        f"?latitude={latitude}&longitude={longitude}"
        f"&current=temperature_2m,wind_speed_10m"
        f"&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data['current']['temperature_2m']  # Just returning temperature

# This tells the LLM about your tool and how to call it
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

# User message asking about the weather in Chennai
input_messages = [{
    "role": "user",
    "content": "What's the weather like in Chennai today?"
}]

# First API call to the model — it will return a tool call if needed
response = client.responses.create(
    model="gpt-4.1-mini",  # or "gpt-4o", "gpt-4.5", etc.
    input=input_messages,
    tools=tools,
)

# Extract the tool call from the response
tool_call = response.output[0]
args = json.loads(tool_call.arguments)  # {"latitude": ..., "longitude": ...}

# Run the function the model wants to call
result = get_weather(args['latitude'], args["longitude"])  # float: temperature

# Now prepare full conversation history
#   → includes the tool call + the result of the tool
input_messages.append(tool_call)  # Add tool_call to the conversation

# Add the result of the tool call, tied to its call_id
input_messages.append({
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)  # Must be string
})

# Second API call — the model now has the function result and gives a final answer
response_2 = client.responses.create(
    model = "gpt-4.1-mini",  # same model as before
    input = input_messages,
    tools = tools
)

# Print the final user-facing message
print(response_2.output[0].content[0].text)


The current temperature in Chennai is approximately 33.7°C today. If you would like more detailed weather information, please let me know!


In [44]:
# Embeddings 

words = [
    "king", "kings", "man", "woman",
    "apple", "banana", "orange", "pear",
    "castle", "throne"
]

# embeddings for the words
response = client.embeddings.create(
    model = 'text-embedding-3-small',
    input = words,
    encoding_format="float"

)

# extract the embedddings 
emeddings = [data.embedding for data in response.data]


In [45]:
print(f"Embedding dimension : {len(emeddings[0])}")
print(f"Number of embeddings : {len(emeddings)}")

Embedding dimension : 1536
Number of embeddings : 10


In [ ]:
import numpy as np
#pip install numpy

# dot product of two vectors
def dot_product(vec1, vec2):
    return np.dot(vec1, vec2)

# similarity maxtrix (dot product b/w all pairs of embeddings)
similarity_matrix = np.zeros((len(words), len(words)))
for i in range(len(words)):
    for j in range(len(words)):
        similarity_matrix[i][j] = dot_product(emeddings[i], emeddings[j])

# print the similarity matrix with labels
print("Similarity Matrix (Dot Products):")
print("         " + " ".join(f"{word:<8}" for word in words))
for i , word in enumerate(words): #enrmuerate used to iterate over the words along with the index
    row_values = " ".join(f"{similarity_matrix[i][j]:.4f}  " for j in range(len(words)))
    print(f"{word:<8} {row_values}")



Similarity Matrix (Dot Products):
         king     kings    man      woman    apple    banana   orange   pear     castle   throne  
king     1.0000   0.5566   0.5179   0.4143   0.2854   0.3167   0.3331   0.3303   0.4565   0.4938  
kings    0.5566   1.0000   0.3095   0.2888   0.2420   0.1786   0.2688   0.2700   0.3337   0.4412  
man      0.5179   0.3095   1.0000   0.7075   0.2748   0.3103   0.3075   0.3601   0.2843   0.2637  
woman    0.4143   0.2888   0.7075   1.0000   0.3281   0.2611   0.3634   0.3922   0.2387   0.2498  
apple    0.2854   0.2420   0.2748   0.3281   1.0000   0.4759   0.4713   0.4770   0.2978   0.2056  
banana   0.3167   0.1786   0.3103   0.2611   0.4759   1.0000   0.4425   0.3694   0.2557   0.2352  
orange   0.3331   0.2688   0.3075   0.3634   0.4713   0.4425   1.0000   0.3792   0.2695   0.2288  
pear     0.3303   0.2700   0.3601   0.3922   0.4770   0.3694   0.3792   1.0000   0.2985   0.2864  
castle   0.4565   0.3337   0.2843   0.2387   0.2978   0.2557   0.2695   0.2

In [47]:
# check for the specific word in the similarity matrix

king_queen_smilarity  = dot_product(emeddings[0], emeddings[1])

In [48]:
print(f"Similarity between 'king' and 'queen': {king_queen_smilarity:.4f}")

Similarity between 'king' and 'queen': 0.5566


In [49]:
print(row_values)

0.4938   0.4412   0.2637   0.2498   0.2056   0.2352   0.2288   0.2864   0.4242   1.0000  
